In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import scipy as sp

# Лабораторная работа №5
## 2 Часть - Задание 4
## Кондрашин Тимофей 211-365 Вариант 2 (11.05.2023)

## Задача и данные

Для выборки значений случайной величины $X$ из генеральной совокупности:
1. Найти выборочные оценки математического ожидания, дисперсии и среднеквадратического отклонения случайной величины X , указать свойства этих оценок.
2. Составить группированный вариационный ряд.
3. Построить гистограмму и полигон относительных частот. На их основе выдвинуть нулевую гипотезу H0 о виде распределения (нормальное распределение).
4. На одном чертеже с гистограммой построить график теоретической плотности вероятностей. Сделать вывод об их визуальном совпадении.
5. Составить эмпирическую функцию распределения $F(x)$ и построить ее график.
6. На одном чертеже с эмпирической функцией распределения построить график теоретической функции распределения. Сделать вывод об их визуальном совпадении.
7. С помощью критерия согласия $\chi^2 $Пирсона проверить гипотезу H0 о виде распределения генеральной совокупности для уровня значимости $a = 0.1$. Сделать статистический вывод.
8. Построить доверительные интервалы для неизвестных математического ожидания и дисперсии нормально распределенной генеральной совокупности с параметрами $a = \overline{x_в}$ и $\sigma = S$ для уровней значимости $a = 0.1, a = 0.05, a = 0.01$. Сделать вывод о ширине доверительного интервала, в зависимости от уровня значимости $a$.

In [65]:
data = pd.DataFrame({'X': [101, 102, 103, 104, 105, 106, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 109, 110, 111, 219, 220, 221, 112, 113, 114, 115, 116, 117, 222, 223, 224, 118, 119, 120, 121, 124, 126, 130, 131, 132]})
alpha_1 = 0.1
alpha_2 = 0.05
alpha_3 = 0.01
n = len(data)

print(f'a_1: {alpha_1}\na_1: {alpha_2}\na_3: {alpha_3}\nКоличество элементов в выборке: {n}')

a_1: 0.1
a_1: 0.05
a_3: 0.01
Количество элементов в выборке: 40


In [66]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
X,101,102,103,104,105,106,208,210,211,212,...,224,118,119,120,121,124,126,130,131,132


## Пункт 1
**Найти выборочные оценки математического ожидания, дисперсии и среднеквадратического отклонения случайной величины X , указать свойства этих оценок.**

### Выборочное среднее для $X$

$$\overline{X} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

Выборочная оценка математического ожидания является несмещенной, так как среднее значение выборки сходится к математическому ожиданию генеральной совокупности.

In [67]:
x_mean = data['X'].mean()
print(f'Выборочное среднее для x: {x_mean:.2f}')

Выборочное среднее для x: 155.55


### Выборочная дисперсия для $X$

$$S^2_x = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{X})^2$$

Оценка дисперсии и среднеквадратического отклонения являются смещенными, так как они рассчитаны по выборке.

In [68]:
x_var = data['X'].var()
print(f'Выборочная дисперсия для x: {x_var:.2f}')

Выборочная дисперсия для x: 2594.41


### Стандартное отклонение для $X$

$$S_x = \sqrt{S^2_x}$$

In [69]:
x_std = data['X'].std()
print(f'Стандартное отклонение для x: {x_std:.2f}')

Стандартное отклонение для x: 50.94


## Пункт 2
**Составить группированный вариационный ряд.**

### Вариационный ряд

In [70]:
data_sort = data['X'].sort_values()
data_sort = data_sort.tolist()
pd.DataFrame(data_sort, columns=['X']).T

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
X,101,102,103,104,105,106,109,110,111,112,...,215,216,217,218,219,220,221,222,223,224


### Группированный вариационный ряд

Сначала необходимо разбить интервальный ряд на интервалы.

Для этого найдём количество интервалов:

$$k = \lceil \log_2 n + 1 \rceil$$

In [71]:
k = int(np.ceil(np.log2(n) + 1))
print(f'Количество интервалов: {k}')

Количество интервалов: 7


Теперь можно разбить интервальный ряд на интервалы:

$$\Delta = \frac{max(x) - min(x)}{k}$$

In [72]:
delta = (data.max()[0] - data.min()[0]) / k
print(f'Длина интервала: {delta:.2f}')

Длина интервала: 17.57


Теперь можно построить интервальный ряд:

In [73]:
intervals = []
for i in range(k):
    intervals.append([data.min()[0] + i * delta, data.min()[0] + (i + 1) * delta])
pd.DataFrame(intervals, columns=['Нижняя граница', 'Верхняя граница'])

,Нижняя граница,Верхняя граница
0,101.000000,118.571429
1,118.571429,136.142857
2,136.142857,153.714286
3,153.714286,171.285714
4,171.285714,188.857143
5,188.857143,206.428571
6,206.428571,224.000000


Далее, необходимо вычислить выборочные частоты попадания значений в каждый интервал:

$$p_i = \frac{n_i}{n}$$

In [74]:
p = []
for i in range(k):
    p.append(len(data[(data['X'] >= 101 + i * delta) & (data['X'] < 101 + (i + 1) * delta)]) / n)
pd.DataFrame(p, columns=['p']).T

,0,1,2,3,4,5,6
p,0.4,0.2,0.0,0.0,0.0,0.0,0.375


Затем нужно вычислить теоретические (ожидаемые) частоты попадания значений в каждый интервал, если бы распределение было нормальным. Для этого воспользуемся формулой:
$$P_i = \Phi \left( \frac{x_{i+1} - \overline{x}}{s} \right) - \Phi \left( \frac{x_{i} - \overline{x}}{s} \right)$$

In [75]:
P = []
for i in range(k):
    P.append(sp.stats.norm.cdf((intervals[i][1] - x_mean) / x_std) - sp.stats.norm.cdf((intervals[i][0] - x_mean) / x_std))
pd.DataFrame(P, columns=['P']).T

,0,1,2,3,4,5,6
P,0.09183,0.117674,0.134029,0.135689,0.122101,0.09766,0.069428


In [76]:
intervals_t = pd.DataFrame(intervals, columns=['Нижняя граница', 'Верхняя граница'])
p_t = pd.DataFrame(p, columns=['p (выборочные частоты)'])
P_t = pd.DataFrame(P, columns=['P (ожидаемые частоты)'])
intervals_t.join(p_t).join(P_t)

,Нижняя граница,Верхняя граница,p (выборочные частоты),P (ожидаемые частоты)
0,101.000000,118.571429,0.400,0.091830
1,118.571429,136.142857,0.200,0.117674
2,136.142857,153.714286,0.000,0.134029
3,153.714286,171.285714,0.000,0.135689
4,171.285714,188.857143,0.000,0.122101
5,188.857143,206.428571,0.000,0.097660
6,206.428571,224.000000,0.375,0.069428


## Пункт 3
**Построить гистограмму и полигон относительных частот. На их основе выдвинуть нулевую гипотезу H0 о виде распределения (нормальное распределение).**|

In [77]:
px.histogram(data, x='X'
             , nbins=10, title='Гистограмма', marginal='rug', height=500)

In [78]:
px.histogram(data, x='X', nbins=10, title='Полигон относительных частот', histnorm='probability density', marginal='rug', height=500)

## Пункт 5
**Составить эмпирическую функцию распределения $F(x)$ и построить ее график**

In [79]:
emp_func = []
for i, x in enumerate(intervals_t['Верхняя граница']):
    emp_func.append(sum(P_t.values.tolist()[0][:i]))
# emp_func.append(1)
emp_func = pd.DataFrame(emp_func).T
emp_func

,0,1,2,3,4,5,6
0,0.0,0.09183,0.09183,0.09183,0.09183,0.09183,0.09183


### График эмпирической функции распределения

$$
F^* (x) =
\begin{cases}
0, && x \le 101 \\
0.09183, && 101.000000 < x \le 118.571429 \\
0.09183, && 118.571429 < x \le 136.142857 \\
0.09183, && 136.142857 < x \le 153.714286 \\
0.09183, && 153.714286 < x \le 171.285714 \\
0.09183, && 171.285714 < x \le 188.857143 \\
0.09183, && 188.857143 < x \le 206.428571 \\
0.09183, && 206.428571 < x \le 224.000000 \\
1, && x > 224.000000
\end{cases}
$$

### График эмпирической функции распределения

In [80]:
px.ecdf(x=emp_func.columns.values.tolist(), y=emp_func.values.tolist()[0], markers=True, ecdfmode="standard",
        title='График эмпирической функции распределения')

In [81]:
px.line(data, x='X', y='F', title='Эмпирическая функция распределения', height=500)


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['X'] but received: F

## Пункт 6
**На одном чертеже с эмпирической функцией распределения построить график теоретической функции распределения. Сделать вывод об их визуальном совпадении.**

## Пункт 7
**С помощью критерия согласия $\chi^2 $Пирсона проверить гипотезу H0 о виде распределения генеральной совокупности для уровня значимости $a = 0.1$. Сделать статистический вывод.**

## Пункт 8
**Построить доверительные интервалы для неизвестных математического ожидания и дисперсии нормально распределенной генеральной совокупности с параметрами $a = \overline{x_в}$ и $\sigma = S$ для уровней значимости $a = 0.1, a = 0.05, a = 0.01$. Сделать вывод о ширине доверительного интервала, в зависимости от уровня значимости $a$.**